In [ ]:
# #for progress bar
# from tqdm import tqdm

''' import libraries '''
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import pycce as pc
import ase
import warnings
from mpl_toolkits import mplot3d
from ase.build import bulk
import pycce.filter
import scipy.stats as stats
from scipy.optimize import curve_fit
import matplotlib as mpl

"""set up seed for bath generations"""
seed = 8805
np.random.seed(seed)
np.set_printoptions(suppress=True, precision=10)

plt.style.use('plot_style.txt')
# mpl.rcParams['figure.figsize'] = [12.0, 8.0]
# mpl.rcParams['xtick.labelsize'] = 15


conc = np.arange(1e20,1.01e21,1e20)
num = np.arange(10,101,10)


#######################################################################################################################
########## depth = 1.9083 nm ##########
#######################################################################################################################
T2_E_near = []
T2_err_E_near = []
for j in conc:
    ''' generate layer of 10-100 electrons within a thin 10x10x1 nm^3 layer '''
    electron_layer = pc.random_bath('e', [1e2, 1e2, 10], 
                                        density=j, density_units='cm-3', 
                                        center=[50,50,60], seed=seed)
    
    ''' generate spin bath of 13C in a 10x10x10 nm^3 diamond lattice '''
    # Generate unitcell from ase
    diamond = pc.read_ase(bulk('C', 'diamond', cubic=True))
    diamond.zdir = [1,1,1]
    
    # Add types of isotopes
    diamond.add_isotopes(('13C', 0.011))
    
    # Add the defect. remove and add atoms at the positions (in cell coordinates) 
    atoms = diamond.gen_supercell(100, remove=[('C', [-5, -5, -5]),('C', [-5+0.5, -5+0.5, -5+0.5])],
                                  add=('14N', [-5+0.5, -5+0.5, -5+0.5]), seed=seed)
    
    """setup for coherence calculation"""
    
    # Parameters of CCE calculations engine
    
    # Order of CCE aproximation
    order = 2
    # Bath cutoff radius
    r_bath = 150  # in A
    # Cluster cutoff radius
    r_dipole = 50  # in A
    
    # Qubit levels (in Sz basis)
    alpha = [0, 0, 1]; beta = [0, 1, 0]
    
    # ZFS Parametters of NV center in diamond
    D = 2.88 * 1e6 # in kHz
    E = 0 # in kHz
    
    # position of central spin
    position = [-5,-5,-5]
    
    # generate central spin
    nv = pc.CenterArray(spin=1, position=position, D=D, E=E, alpha=alpha, beta=beta)
    
    # Setting the runner engine
    calc = pc.Simulator(spin=nv, bath=np.concatenate([atoms,electron_layer]),
                        r_bath=r_bath, r_dipole=r_dipole, order=order)
    
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    "coherence calculation"
    
    "general"
    # Time points
    time_space = np.linspace(0, 0.1, 201)  # in ms
    
    # Mag. Field (Bx By Bz)
    b = np.array([0,0,500])  # in G
    
    # Hahn-echo pulse sequence
    pulse_sequence = [pc.Pulse('x', np.pi)]
    
    # Calculate coherence function with general method
    l_generatilze = calc.compute(time_space, magnetic_field=b,
                                 pulses=pulse_sequence,
                                 method='gcce', quantity='coherence')
    
    ''' filter faulty points '''
    l_generatilze_test = l_generatilze.real
    
    time_space_generatilze_fixed = []
    l_generatilze_fixed = []
    for i in np.arange(0,201):
        if l_generatilze_test[i] < 1.25:
            l_generatilze_fixed.append(l_generatilze_test[i])
            time_space_generatilze_fixed.append(time_space[i])
    
    
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    "fitting a negative squared exponential"
    
    #popt: Optimized parameters
    #pcov: matrix of covariance.
    model = np.polyfit(time_space_generatilze_fixed, np.log(abs(l_generatilze_fixed)), 2, full=False, cov=True)
    model_T2 = 1/np.sqrt(abs(model[0][0]))
    model_err = np.sqrt(model[1][0][0])
    T2_err = (model_err/model[0][0])/2
    
    T2_E_near.append(np.abs(T2_gen[0]))
    T2_err_E_near.append(T2_var[0][0])
    
#######################################################################################################################
########## depth = 5.0 nm ##########
#######################################################################################################################
T2_E_mid = []
T2_err_E_mid = []
for j in conc:
    ''' generate layer of 10-100 electrons within a thin 10x10x1 nm^3 layer '''
    electron_layer = pc.random_bath('e', [1e2, 1e2, 10], 
                                        density=j, density_units='cm-3', 
                                        center=[50,50,60], seed=seed)
    
    ''' generate spin bath of 13C in a 10x10x10 nm^3 diamond lattice '''
    # Generate unitcell from ase
    diamond = pc.read_ase(bulk('C', 'diamond', cubic=True))
    diamond.zdir = [1,1,1]
    
    # Add types of isotopes
    diamond.add_isotopes(('13C', 0.011))
    
    # Add the defect. remove and add atoms at the positions (in cell coordinates) 
    atoms = diamond.gen_supercell(100, remove=[('C', [0, 0, 0]),('C', [0+0.5, 0+0.5, 0+0.5])],
                                  add=('14N', [0+0.5, 0+0.5, 0+0.5]), seed=seed)
    
    """setup for coherence calculation"""
    
    # Parameters of CCE calculations engine
    
    # Order of CCE aproximation
    order = 2
    # Bath cutoff radius
    r_bath = 150  # in A
    # Cluster cutoff radius
    r_dipole = 50  # in A
    
    # Qubit levels (in Sz basis)
    alpha = [0, 0, 1]; beta = [0, 1, 0]
    
    # ZFS Parametters of NV center in diamond
    D = 2.88 * 1e6 # in kHz
    E = 0 # in kHz
    
    # position of central spin
    position = [0,0,0]
    
    # generate central spin
    nv = pc.CenterArray(spin=1, position=position, D=D, E=E, alpha=alpha, beta=beta)
    
    # Setting the runner engine
    calc = pc.Simulator(spin=nv, bath=np.concatenate([atoms,electron_layer]),
                        r_bath=r_bath, r_dipole=r_dipole, order=order)
    
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    "coherence calculation"
    
    "general"
    # Time points
    time_space = np.linspace(0, 1, 201)  # in ms
    
    # Mag. Field (Bx By Bz)
    b = np.array([0,0,500])  # in G
    
    # Hahn-echo pulse sequence
    pulse_sequence = [pc.Pulse('x', np.pi)]
    
    # Calculate coherence function with general method
    l_generatilze = calc.compute(time_space, magnetic_field=b,
                                 pulses=pulse_sequence,
                                 method='gcce', quantity='coherence')
    
    ''' filter faulty points '''
    l_generatilze_test = l_generatilze.real
    
    time_space_generatilze_fixed = []
    l_generatilze_fixed = []
    for i in np.arange(0,201):
        if l_generatilze_test[i] < 1.25:
            l_generatilze_fixed.append(l_generatilze_test[i])
            time_space_generatilze_fixed.append(time_space[i])
    
    
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    "fitting a negative squared exponential"
    def coherence_fit(x,T2):
        "fit coherence function to a negative squared exponential function with time constant T2"
        fit = np.exp(-(x/T2)**2)
        return fit
    
    #popt: Optimized parameters
    #pcov: matrix of covariance.
    T2_gen,T2_var = curve_fit(coherence_fit,time_space_generatilze_fixed, l_generatilze_fixed)
    T2_var = np.sqrt(np.abs(T2_var))
    
    T2_E_mid.append(np.abs(T2_gen[0]))
    T2_err_E_mid.append(T2_var[0][0])

#######################################################################################################################
########## depth = 8.0917 nm ##########
#######################################################################################################################
T2_E_far = []
T2_err_E_far = []
for j in conc:
    ''' generate layer of 10-100 electrons within a thin 10x10x1 nm^3 layer '''
    electron_layer = pc.random_bath('e', [1e2, 1e2, 10], 
                                        density=j, density_units='cm-3', 
                                        center=[50,50,60], seed=seed)
    
    ''' generate spin bath of 13C in a 10x10x10 nm^3 diamond lattice '''
    # Generate unitcell from ase
    diamond = pc.read_ase(bulk('C', 'diamond', cubic=True))
    diamond.zdir = [1,1,1]
    
    # Add types of isotopes
    diamond.add_isotopes(('13C', 0.011))
    
    # Add the defect. remove and add atoms at the positions (in cell coordinates) 
    atoms = diamond.gen_supercell(100, remove=[('C', [5, 5, 5]),('C', [5+0.5, 5+0.5, 5+0.5])],
                                  add=('14N', [5+0.5, 5+0.5, 5+0.5]), seed=seed)
    
    """setup for coherence calculation"""
    
    # Parameters of CCE calculations engine
    
    # Order of CCE aproximation
    order = 2
    # Bath cutoff radius
    r_bath = 150  # in A
    # Cluster cutoff radius
    r_dipole = 50  # in A
    
    # Qubit levels (in Sz basis)
    alpha = [0, 0, 1]; beta = [0, 1, 0]
    
    # ZFS Parametters of NV center in diamond
    D = 2.88 * 1e6 # in kHz
    E = 0 # in kHz
    
    # position of central spin
    position = [5,5,5]
    
    # generate central spin
    nv = pc.CenterArray(spin=1, position=position, D=D, E=E, alpha=alpha, beta=beta)
    
    # Setting the runner engine
    calc = pc.Simulator(spin=nv, bath=np.concatenate([atoms,electron_layer]),
                        r_bath=r_bath, r_dipole=r_dipole, order=order)
    
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    "coherence calculation"
    
    "general"
    # Time points
    time_space = np.linspace(0, 1, 201)  # in ms
    
    # Mag. Field (Bx By Bz)
    b = np.array([0,0,500])  # in G
    
    # Hahn-echo pulse sequence
    pulse_sequence = [pc.Pulse('x', np.pi)]
    
    # Calculate coherence function with general method
    l_generatilze = calc.compute(time_space, magnetic_field=b,
                                 pulses=pulse_sequence,
                                 method='gcce', quantity='coherence')
    
    ''' filter faulty points '''
    l_generatilze_test = l_generatilze.real
    
    time_space_generatilze_fixed = []
    l_generatilze_fixed = []
    for i in np.arange(0,201):
        if l_generatilze_test[i] < 1.25:
            l_generatilze_fixed.append(l_generatilze_test[i])
            time_space_generatilze_fixed.append(time_space[i])
    
    
    ''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    "fitting a negative squared exponential"
    def coherence_fit(x,T2):
        "fit coherence function to a negative squared exponential function with time constant T2"
        fit = np.exp(-(x/T2)**2)
        return fit
    
    #popt: Optimized parameters
    #pcov: matrix of covariance.
    T2_gen,T2_var = curve_fit(coherence_fit,time_space_generatilze_fixed, l_generatilze_fixed)
    T2_var = np.sqrt(np.abs(T2_var))
    
    T2_E_far.append(np.abs(T2_gen[0]))
    T2_err_E_far.append(T2_var[0][0])

# plots the data points and the fitted  curve
plt.figure()
plt.errorbar(num, T2_E_near, yerr=T2_err_E_near, fmt='^', ecolor='k', elinewidth=1, capsize=5, label='1.9083 nm')
plt.errorbar(num, T2_E_mid , yerr=T2_err_E_mid , fmt='o', ecolor='k', elinewidth=1, capsize=5, label='5.0 nm')
plt.errorbar(num, T2_E_far , yerr=T2_err_E_far , fmt='s', ecolor='k', elinewidth=1, capsize=5, label='8.0917 nm')
plt.legend()
plt.xlabel('Number of electrons')
plt.ylabel('$T_2$ (ms)')

datasavefile = '/home/xzcapxpl/electron_surface/plot/datapoints_T2_E.csv'
data_E_T2 = {'num':num, 'T2_E_near':T2_E_near, 'T2_err_E_near':T2_err_E_near, 
                        'T2_E_mid':T2_E_mid, 'T2_err_E_mid':T2_err_E_mid, 
                        'T2_E_far':T2_E_far, 'T2_err_E_far':T2_err_E_far}

df = pd.DataFrame(data = data_E_T2)
df.to_csv(datasavefile)

outpath = '/home/xzcapxpl/electron_surface/plot/atomnumb_T2_E.png'
plt.savefig(outpath)